### Importing Necessary Files


In [1]:
import os
import random
import time
import argparse
from datetime import timezone

import yaml
import psycopg2
from psycopg2 import sql
from psycopg2.extras import execute_values
from faker import Faker

In [ ]:
# ---------- Configuration loading ----------
def load_config(yaml_path="dbconfig.yaml"):
    cfg = {}
    if os.path.exists(yaml_path):
        with open(yaml_path, "r", encoding="utf-8") as f:
            y = yaml.safe_load(f) or {}